In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import math
import random
from random import randrange
import matplotlib.pyplot as plt
import pickle

In [ ]:
class Graphs():
    def __init__(self, random_seed = 12345):
        random.seed(random_seed)
        
        self.G = nx.Graph()
        
        self.N = 0
        self.m = 0
        self.m0 = 0
        self.p = 0

        
    def barabasi_albert(self, n_new):
        # tworze liste prawdopodobienstw tego, że nowy wierzchołek dołączy się do starego
        n_edges = self.G.number_of_edges()
        n_nodes = len(self.G.nodes())
        p = np.zeros(n_nodes)
        
        for i,n in enumerate(self.G.nodes()):
            p[i] = len(self.G.edges(n))/n_edges

        p = p/p.sum()
        
        # wybieram m wierzchołków (bez zwracania) z listy wszystkich 
        # wierzchołkow z policzonym wcześniej prawdopodobienstwem
        n_old = np.random.choice(range(n_nodes), p=p, size=self.m, replace=False)
        
        # dodaje 
        for n_o in n_old:
            self.G.add_edge(n_new, n_o)
            
    
    def barabasi_albert_matrix(self):
        '''nieudana próba optymalizacji'''
        adjMatrix = np.zeros((self.N,self.N))
        
        for i in range(self.m0):
            for j in range(self.m0):
                adjMatrix[i][j] = 1
                adjMatrix[j][i] = 1
                
            adjMatrix[i][i] = 0
        
        
        for i in range(self.m0+1,self.N):
            p = np.zeros(i)
            
            for i_p in range(0,i):
                p[i_p] = 2*adjMatrix[i_p].sum()/adjMatrix.sum()
            
            p = p/p.sum()
            
            n_old = np.random.choice(range(i), p=p, size=self.m, replace=False)
            
            for n in n_old:
                adjMatrix[i][n] = 1
                adjMatrix[n][i] = 1
                
        self.G = nx.from_numpy_matrix(adjMatrix)
        
        return self.G
            
            
    def erdos_renyi(self):
        adjMatrix = np.zeros((self.N,self.N))
        
        for i in range(0, self.N):
            for j in range(0, i):
                if random.uniform(0, 1) <= self.p:
                    adjMatrix[i][j] = 1
                    adjMatrix[j][i] = 1
    
        self.G = nx.from_numpy_matrix(adjMatrix)

        
    def create(self, graph, N=0, m=0, m0=0, k_mean=0):
        valid = {'barabasi_albert', 'erdos_renyi'}
        if graph not in valid:
            raise ValueError("results: graph must be one of %r." % valid)
        
        
        if graph == 'barabasi_albert':
            self.N = N
            self.m = m
            self.m0 = m0
            
            #self.barabasi_albert_matrix()
            
            self.G = nx.complete_graph(self.m0)
            for i in range(0, self.N):
                self.barabasi_albert(i)
                
        if graph == 'erdos_renyi':
            self.N = N
            self.p = k_mean/(N-1)
            
            self.erdos_renyi()
                
        return self.G

In [ ]:
class Model():
    def __init__(self, Graph, a, b, R, n_iter):
        self.G = Graph
        self.G_size = len(self.G.nodes())
        self.a = a
        self.b = b
        self.R = R
        self.n_iter = n_iter
        
        self.N_ill = []
        
        
    def initial_state(self):
        attrs = {}
        for n in self.G.nodes():
            attrs[n] = {"state": 0}
        
        nx.set_node_attributes(self.G, attrs)
        
    
    def patients_zero_generate(self):
        patients_zero = np.random.choice(list(self.G.nodes()), size=self.R, replace=False)
        
        attrs = {}
        for n in patients_zero:
            attrs[n] = {"state": 1}
        
        nx.set_node_attributes(self.G, attrs)
            
    def simulate(self):
        self.initial_state()
        self.patients_zero_generate()
        
        for i in range(self.n_iter):
            r_node = randrange(self.G_size)
            
            if self.G.nodes[r_node]['state'] == 1:
                if random.uniform(0, 1) <= self.a:
                    self.G.nodes[r_node]["state"] = 0
                else:
                    pass
            
            if self.G.nodes[r_node]['state'] == 0:
                r_node_neighbors = [n for n in self.G[r_node]]
                try:
                    """neighbor = np.random.choice(r_node_neighbors, size=1)[0]
                
                    if self.G.nodes[neighbor]['state'] == 1:
                        if random.uniform(0, 1) <= self.b:
                            self.G.nodes[r_node]["state"] = 1
                        else:
                            pass
                    else:
                        pass"""
                    for n in r_node_neighbors:
                        if self.G.nodes[n]['state'] == 1:
                            if random.uniform(0, 1) <= self.b:
                                self.G.nodes[r_node]["state"] = 1
                        else:
                            pass
                    else:
                        pass
                except:
                    pass
                
            N1 = len([x for x,y in self.G.nodes(data=True) if y['state']==1])
            self.N_ill.append(N1)

In [ ]:
G = Graphs()

In [ ]:
barabasi_albert=G.create('barabasi_albert', N=10000, m=2, m0=2)

In [ ]:
erdos_renyi=G.create('erdos_renyi', N=10000, k_mean=4)

# Wartości teoretyczne b/a

In [ ]:
lambda_c_ER = 1/4

In [ ]:
lambda_c_ER

In [ ]:
lambda_c_BA = 2/(2*np.log(10000))

In [ ]:
lambda_c_BA

# Symulacje

## N(R)

#### lambda > lambda_c

In [ ]:
R = [50, 100, 1000, 5000, 9000]

sim_ER = []
#sim_BA = []

for r in R:
    model_ER = Model(Graph=erdos_renyi, a=0.4, b=0.2, R=r, n_iter=300000)
    model_ER.simulate()
    sim_ER.append(model_ER.N_ill)
    
    #model_BA = Model(Graph=barabasi_albert, a=0.4, b=0.2, R=r, n_iter=200000)
    #model_BA.simulate()
    #sim_BA.append(model_BA.N_ill)
    
    print(r)

In [ ]:
with open('sim_ER_v2.pkl', 'wb') as f:
    pickle.dump(sim_ER, f)
    
with open('sim_BA_v2.pkl', 'wb') as f:
    pickle.dump(sim_BA, f)

In [ ]:
with open('sim_ER_v2.pkl', 'rb') as f:
    sim_ER = pickle.load(f)
    
with open('sim_BA_v2.pkl', 'rb') as f:
    sim_BA = pickle.load(f)

#### lambda < lambda_c

In [ ]:
R = [50, 100, 1000, 5000, 9000]

sim_ER_2 = []
sim_BA_2 = []

for r in R:
    model_ER = Model(Graph=erdos_renyi, a=0.7, b=0.1, R=r, n_iter=1000000)
    model_ER.simulate()
    sim_ER_2.append(model_ER.N_ill)
    
    model_BA = Model(Graph=barabasi_albert, a=0.7, b=0.05, R=r, n_iter=1000000)
    model_BA.simulate()
    sim_BA_2.append(model_BA.N_ill)
    
    print(r)

In [ ]:
with open('sim_ER_2.pkl', 'wb') as f:
    pickle.dump(sim_ER_2, f)
    
with open('sim_BA_2.pkl', 'wb') as f:
    pickle.dump(sim_BA_2, f)

In [ ]:
with open('sim_ER_2.pkl', 'rb') as f:
    sim_ER_2 = pickle.load(f)
    
with open('sim_BA_2.pkl', 'rb') as f:
    sim_BA_2 = pickle.load(f)

### N(lambda)

In [ ]:
ba = [(0.02, 0.7), (0.05, 0.7), (0.06, 0.7), (0.07, 0.7), (0.08, 0.7), (0.09, 0.7), (0.1, 0.7), 
      (0.11, 0.7), (0.12, 0.7), (0.13, 0.7), (0.14, 0.7), (0.15, 0.7), (0.16, 0.7),
      (0.17, 0.7), (0.18, 0.7), (0.19, 0.7), (0.20, 0.7), (0.21, 0.7), (0.22, 0.7), 
      (0.23, 0.7), (0.24, 0.7), (0.03, 0.7), (0.04, 0.7)]

In [ ]:
sim_ER_3 = []
sim_BA_3 = []

for i in ba2:
    model_ER = Model(Graph=erdos_renyi, a=i[1], b=i[0], R=1000, n_iter=200000)
    model_ER.simulate()
    sim_ER_3.append(model_ER.N_ill)
    
    model_BA = Model(Graph=barabasi_albert, a=i[1], b=i[0], R=1000, n_iter=200000)
    model_BA.simulate()
    sim_BA_3.append(model_BA.N_ill)
    
    print(i[0]/i[1])

In [ ]:
sim_ER_3_final = [i[-1] for i in sim_ER_3]

sim_BA_3_final = [i[-1] for i in sim_BA_3]

In [ ]:
with open('sim_ER_3_final.pkl', 'wb') as f:
    pickle.dump(sim_ER_3_final, f)
    
with open('sim_BA_3_final.pkl', 'wb') as f:
    pickle.dump(sim_BA_3_final, f)

In [ ]:
with open('sim_ER_3_final.pkl', 'rb') as f:
    sim_ER_3_final = pickle.load(f)
    
with open('sim_BA_3_final.pkl', 'rb') as f:
    sim_BA_3_final = pickle.load(f)

# Wizualizacja wyników

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(np.array(sim_ER[0])[:300000]/10000, linewidth=3, label="R = 50")
plt.plot(np.array(sim_ER[1])[:300000]/10000, linewidth=3, label="R = 100")
plt.plot(np.array(sim_ER[2])[:300000]/10000, linewidth=3, label="R = 1000")
plt.plot(np.array(sim_ER[3])[:300000]/10000, linewidth=3, label="R = 5000")
plt.plot(np.array(sim_ER[4])[:300000]/10000, linewidth=3, label="R = 9000")
plt.xlabel('t', size=25)
plt.ylabel('$\\frac{N_1}{N}$', size=25, rotation=0, labelpad=15)
plt.ylim([0,1])
plt.title('SIS dla grafu Erdős–Rényi, $\lambda = 0.5$', size=30)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(loc=4, prop={'size': 20})

plt.plot([-10000,310000],[0.5,0.5], '--', c='r')
plt.xlim([-10000,310000])

plt.savefig('ER_05.svg')
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(np.array(sim_BA[0])[:200000]/10000, linewidth=3, label="R = 50")
plt.plot(np.array(sim_BA[1])[:200000]/10000, linewidth=3, label="R = 100")
plt.plot(np.array(sim_BA[2])[:200000]/10000, linewidth=3, label="R = 1000")
plt.plot(np.array(sim_BA[3])[:200000]/10000, linewidth=3, label="R = 5000")
plt.plot(np.array(sim_BA[4])[:200000]/10000, linewidth=3, label="R = 9000")
plt.ylim((0,1))
plt.xlabel('t', size=25)
plt.ylabel('$\\frac{N_{1}}{N}$', size=25, rotation=0, labelpad=15)
plt.title('SIS dla grafu Barabási–Albert, $\lambda = 0.5$', size=30)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(loc=4, prop={'size': 20})

plt.plot([-10000,210000],[0.5,0.5], '--', c='r')
plt.xlim([-10000,210000])

plt.savefig('BA_05.svg')
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(np.array(sim_ER_2[0])[:200000]/10000, linewidth=3, label="R = 50")
plt.plot(np.array(sim_ER_2[1])[:200000]/10000, linewidth=3, label="R = 100")
plt.plot(np.array(sim_ER_2[2])[:200000]/10000, linewidth=3, label="R = 1000")
plt.plot(np.array(sim_ER_2[3])[:200000]/10000, linewidth=3, label="R = 5000")
plt.plot(np.array(sim_ER_2[4])[:200000]/10000, linewidth=3, label="R = 9000")
plt.ylim((0,1))
plt.xlabel('t', size=25)
plt.ylabel('$\\frac{N_{1}}{N}$', size=25, rotation=0, labelpad=15)
plt.title('SIS dla grafu Erdős–Rényi, $\lambda \\approx 0.14$', size=30)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(loc=1, prop={'size': 20})

plt.savefig('ER_014.svg')
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(np.array(sim_BA_2[0])[:200000]/10000, linewidth=3, label="R = 50")
plt.plot(np.array(sim_BA_2[1])[:200000]/10000, linewidth=3, label="R = 100")
plt.plot(np.array(sim_BA_2[2])[:200000]/10000, linewidth=3, label="R = 1000")
plt.plot(np.array(sim_BA_2[3])[:200000]/10000, linewidth=3, label="R = 5000")
plt.plot(np.array(sim_BA_2[4])[:200000]/10000, linewidth=3, label="R = 9000")
plt.ylim((0,1))
plt.xlabel('t', size=25)
plt.ylabel('$\\frac{N_{1}}{N}$', size=25, rotation=0, labelpad=15)
plt.title('SIS dla grafu Barabási–Albert, $\lambda \\approx 0.07$', size=30)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(loc=1, prop={'size': 20})

plt.savefig('BA_007.svg')
plt.show()

In [ ]:
x = [i[0]/i[1] for i in ba]

In [ ]:
plt.figure(figsize=(14,7))
plt.scatter(x,np.array(sim_ER_3_final)/10000)
plt.xlabel('$\lambda$', size=25)
plt.ylabel('$\\frac{N_{1}}{N}$', size=25, rotation=0, labelpad=15)
plt.title('SIS dla grafu Erdős–Rényi, w zależności od $\lambda$', size=30)
plt.tick_params(axis='both', which='major', labelsize=16)

plt.plot([0.25,0.25],[-1,1], '--', c='r')
plt.plot([0.20714285714285718,0.20714285714285718],[-1,1], '--', c='b')
plt.ylim([-0.05,0.5])

plt.savefig('ER_lam.svg')
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
plt.scatter(x,np.array(sim_BA_3_final)/10000)
plt.xlabel('$\lambda$', size=25)
plt.ylabel('$\\frac{N_{1}}{N}$', size=25, rotation=0, labelpad=15)
plt.title('SIS dla grafu Barabási–Albert, w zależności od $\lambda$', size=30)
plt.tick_params(axis='both', which='major', labelsize=16)

plt.plot([2/(2*np.log(10000)),2/(2*np.log(10000))],[-1,1], '--', c='r')
plt.plot([(0.10000000000000002+0.08571428571428572)/2,(0.10000000000000002+0.08571428571428572)/2],[-1,1], '--', c='b')
plt.ylim([-0.05,0.5])

plt.savefig('BA_lam.svg')
plt.show()